In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../input/shopee-product-matching/train.csv")

In [3]:
df["label_group"].value_counts()

994676122     51
159351600     51
562358068     51
3113678103    51
1163569239    51
              ..
3996065489     2
517023446      2
2101581536     2
3460538113     2
3927456083     2
Name: label_group, Length: 11014, dtype: int64

In [4]:
df["phash_size"] = df["image_phash"].map(df["image_phash"].value_counts().to_dict())
df["label_size"] = df["label_group"].map(df["label_group"].value_counts().to_dict())

In [5]:
df[df["label_size"] > 1].sort_values("label_group")

,posting_id,image,image_phash,title,label_group,phash_size,label_size
3874,train_1646767365,1d7aadc7503b2b4539cc9a5fe41979dd.jpg,e925873ed09cd08f,Sarung celana wadimor original 100% dewasa dan...,258047,1,3
31859,train_1528423085,eec692257e74fcbc6cb63cb76d0f20e7.jpg,ea97861c926a71e3,WARNA RANDOM ACAK Sarung Celana Wadimor MURAH ...,258047,1,3
6738,train_398181303,3301b8aaccea93d1098995ffbc537335.jpg,e9b5833e929e909c,SARUNG CELANA WADIMOR DEWASA HITAM POLOS SARCEL,258047,1,3
7613,train_2865605743,3977f4e7a47c73568c5e9fcb61723b4b.jpg,bfc3cc1cc636c14c,Wall Sticker / WallSticker - Submarine Measur...,297977,1,2
12367,train_1382500866,5d075d7eaa258052ab125c75c06293d6.jpg,838436c07dff19e4,RELIZA WALL STICKER PENGUKUR TINGGI BADAN JERA...,297977,1,2
...,...,...,...,...,...,...,...
5738,train_3671173710,2b2680539685bd6adada6940e443a1d5.jpg,abb51a9b04ca6597,Earphone Gaming Plextone G20 Game Handsfree,4293276364,1,2
4554,train_264319412,2254f0b3b2c80b678b41dcd588d77320.jpg,c3cc3c332c7333cc,Baca Buku Ini Saat Engkau Patah Hati : Sesungg...,4294197112,1,4
6655,train_211794714,325d2bec9a79fa966472229ea0d0b28e.jpg,e3ccc9338c33978c,Baca Buku ini Saat Engkau Patah Hati - Anak Hebat,4294197112,1,4
33503,train_3056056584,fa804f0fdf85e2497640ee48ad4bc3d1.jpg,ea93956c934a7ca1,BACA BUKU INI SAAT ENGKAU PATAH HATI: SESUNGGU...,4294197112,1,4


In [35]:
df["fold"] = df["label_group"] % 5

In [36]:
df.groupby("fold")["label_size"].describe()

,count,mean,std,min,25%,50%,75%,max
fold,,,,,,,,
0,7148.0,6.607163,8.956879,2.0,2.0,3.0,6.0,51.0
1,6754.0,5.975718,8.233649,2.0,2.0,3.0,6.0,51.0
2,6820.0,4.978886,6.320623,2.0,2.0,3.0,5.0,51.0
3,6849.0,5.938823,8.111747,2.0,2.0,3.0,6.0,51.0
4,6679.0,5.919000,7.770915,2.0,2.0,3.0,6.0,51.0


In [46]:
import tqdm

df_all = []
i = 0
for label_size in np.arange(2, 52):
    for _, w_df in df[df["label_size"] == label_size].groupby("label_group"):
        w_df["fold"] = i % 5
        i += 1
        df_all.append(w_df)

In [47]:
df_all = pd.concat(df_all)

In [48]:
# label_size均衡
df_all.groupby("fold")["label_size"].describe()

,count,mean,std,min,25%,50%,75%,max
fold,,,,,,,,
0,6839.0,5.797485,7.667869,2.0,2.0,3.0,6.0,51.0
1,6849.0,5.874580,7.900107,2.0,2.0,3.0,6.0,51.0
2,6860.0,5.969679,8.187389,2.0,2.0,3.0,6.0,51.0
3,6872.0,6.048894,8.396394,2.0,2.0,3.0,6.0,51.0
4,6830.0,5.761054,7.584715,2.0,2.0,3.0,6.0,51.0


In [49]:
df_all.to_csv("../input/shopee-product-matching/train_fold.csv", index=False)